# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Aug 10 2022 2:44PM,243907,21,623795,"NBTY Global, Inc.",Released
1,Aug 10 2022 2:39PM,243906,20,ALU0007511,Alumier Labs Inc.,Released
2,Aug 10 2022 2:36PM,243905,19,ACG-2102490912,ACG North America LLC,Released
3,Aug 10 2022 2:33PM,243904,19,ACG-2102490913,ACG North America LLC,Released
4,Aug 10 2022 2:27PM,243903,19,60017870,"GUSA Granules USA, Inc.",Released
5,Aug 10 2022 2:25PM,243902,10,CTF00000977,"Citieffe, Inc.",Released
6,Aug 10 2022 2:17PM,243901,21,623782,"NBTY Global, Inc.",Released
7,Aug 10 2022 2:13PM,243900,21,623781,"NBTY Global, Inc.",Released
8,Aug 10 2022 2:06PM,243899,20,8384543,"Exact-Rx, Inc.",Released
9,Aug 10 2022 2:06PM,243899,20,8385777,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,243903,Released,1
39,243904,Released,1
40,243905,Released,1
41,243906,Released,1
42,243907,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
243903,NaN,NaN,1.0
243904,NaN,NaN,1.0
243905,NaN,NaN,1.0
243906,NaN,NaN,1.0
243907,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243808,36.0,18.0,1.0
243811,0.0,0.0,3.0
243828,0.0,1.0,0.0
243847,0.0,1.0,19.0
243860,1.0,22.0,13.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
243808,36,18,1
243811,0,0,3
243828,0,1,0
243847,0,1,19
243860,1,22,13


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243808,36,18,1
1,243811,0,0,3
2,243828,0,1,0
3,243847,0,1,19
4,243860,1,22,13


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,243808,36,18,1
1,243811,,,3
2,243828,,1,
3,243847,,1,19
4,243860,1,22,13


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Aug 10 2022 2:44PM,243907,21,"NBTY Global, Inc."
1,Aug 10 2022 2:39PM,243906,20,Alumier Labs Inc.
2,Aug 10 2022 2:36PM,243905,19,ACG North America LLC
3,Aug 10 2022 2:33PM,243904,19,ACG North America LLC
4,Aug 10 2022 2:27PM,243903,19,"GUSA Granules USA, Inc."
5,Aug 10 2022 2:25PM,243902,10,"Citieffe, Inc."
6,Aug 10 2022 2:17PM,243901,21,"NBTY Global, Inc."
7,Aug 10 2022 2:13PM,243900,21,"NBTY Global, Inc."
8,Aug 10 2022 2:06PM,243899,20,"Exact-Rx, Inc."
10,Aug 10 2022 2:06PM,243898,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Aug 10 2022 2:44PM,243907,21,"NBTY Global, Inc.",,,1
1,Aug 10 2022 2:39PM,243906,20,Alumier Labs Inc.,,,1
2,Aug 10 2022 2:36PM,243905,19,ACG North America LLC,,,1
3,Aug 10 2022 2:33PM,243904,19,ACG North America LLC,,,1
4,Aug 10 2022 2:27PM,243903,19,"GUSA Granules USA, Inc.",,,1
5,Aug 10 2022 2:25PM,243902,10,"Citieffe, Inc.",,,1
6,Aug 10 2022 2:17PM,243901,21,"NBTY Global, Inc.",,,1
7,Aug 10 2022 2:13PM,243900,21,"NBTY Global, Inc.",,,1
8,Aug 10 2022 2:06PM,243899,20,"Exact-Rx, Inc.",,,2
9,Aug 10 2022 2:06PM,243898,10,ISDIN Corporation,,,28


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 2:44PM,243907,21,"NBTY Global, Inc.",1,,
1,Aug 10 2022 2:39PM,243906,20,Alumier Labs Inc.,1,,
2,Aug 10 2022 2:36PM,243905,19,ACG North America LLC,1,,
3,Aug 10 2022 2:33PM,243904,19,ACG North America LLC,1,,
4,Aug 10 2022 2:27PM,243903,19,"GUSA Granules USA, Inc.",1,,
5,Aug 10 2022 2:25PM,243902,10,"Citieffe, Inc.",1,,
6,Aug 10 2022 2:17PM,243901,21,"NBTY Global, Inc.",1,,
7,Aug 10 2022 2:13PM,243900,21,"NBTY Global, Inc.",1,,
8,Aug 10 2022 2:06PM,243899,20,"Exact-Rx, Inc.",2,,
9,Aug 10 2022 2:06PM,243898,10,ISDIN Corporation,28,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 2:44PM,243907,21,"NBTY Global, Inc.",1,,
1,Aug 10 2022 2:39PM,243906,20,Alumier Labs Inc.,1,,
2,Aug 10 2022 2:36PM,243905,19,ACG North America LLC,1,,
3,Aug 10 2022 2:33PM,243904,19,ACG North America LLC,1,,
4,Aug 10 2022 2:27PM,243903,19,"GUSA Granules USA, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 2:44PM,243907,21,"NBTY Global, Inc.",1.0,NaN,NaN
1,Aug 10 2022 2:39PM,243906,20,Alumier Labs Inc.,1.0,NaN,NaN
2,Aug 10 2022 2:36PM,243905,19,ACG North America LLC,1.0,NaN,NaN
3,Aug 10 2022 2:33PM,243904,19,ACG North America LLC,1.0,NaN,NaN
4,Aug 10 2022 2:27PM,243903,19,"GUSA Granules USA, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Aug 10 2022 2:44PM,243907,21,"NBTY Global, Inc.",1.0,0.0,0.0
1,Aug 10 2022 2:39PM,243906,20,Alumier Labs Inc.,1.0,0.0,0.0
2,Aug 10 2022 2:36PM,243905,19,ACG North America LLC,1.0,0.0,0.0
3,Aug 10 2022 2:33PM,243904,19,ACG North America LLC,1.0,0.0,0.0
4,Aug 10 2022 2:27PM,243903,19,"GUSA Granules USA, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1951024,47.0,16.0,8.0
15,731587,21.0,22.0,1.0
16,243865,1.0,0.0,0.0
19,2438904,22.0,4.0,0.0
20,1219331,37.0,19.0,36.0
21,1219433,4.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1951024,47.0,16.0,8.0
1,15,731587,21.0,22.0,1.0
2,16,243865,1.0,0.0,0.0
3,19,2438904,22.0,4.0,0.0
4,20,1219331,37.0,19.0,36.0
5,21,1219433,4.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,47.0,16.0,8.0
1,15,21.0,22.0,1.0
2,16,1.0,0.0,0.0
3,19,22.0,4.0,0.0
4,20,37.0,19.0,36.0
5,21,4.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,47.0
1,15,Released,21.0
2,16,Released,1.0
3,19,Released,22.0
4,20,Released,37.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,8.0,1.0,0.0,0.0,36.0,0.0
Executing,16.0,22.0,0.0,4.0,19.0,1.0
Released,47.0,21.0,1.0,22.0,37.0,4.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,8.0,1.0,0.0,0.0,36.0,0.0
1,Executing,16.0,22.0,0.0,4.0,19.0,1.0
2,Released,47.0,21.0,1.0,22.0,37.0,4.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,8.0,1.0,0.0,0.0,36.0,0.0
1,Executing,16.0,22.0,0.0,4.0,19.0,1.0
2,Released,47.0,21.0,1.0,22.0,37.0,4.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()